# データの前処理
対話文のデータセットに前処理を行い、保存します。

## 対話文の取得
雑談対話コーパス「projectnextnlp-chat-dialogue-corpus.zip」をダウンロードします。  
  
> Copyright (c) 2015 Project Next NLP 対話タスク 参加者一同  
> https://sites.google.com/site/dialoguebreakdowndetection/chat-dialogue-corpus/LICENSE.txt  
> Released under the MIT license
 
フォルダからjsonファイルを読み込み、対話文として成り立っている文章を取り出してリストに格納します。  



In [2]:
import glob  # ファイルの取得に使用
import json  # jsonファイルの読み込みに使用
import re

path = "../section_4/projectnextnlp-chat-dialogue-corpus/json"  # フォルダの場所を指定

files = glob.glob(path + "/*/*.json")  # ファイルの一覧
files[:10]

['../section_4/projectnextnlp-chat-dialogue-corpus/json/init100/1408695471.log.json',
 '../section_4/projectnextnlp-chat-dialogue-corpus/json/init100/1409463385.log.json',
 '../section_4/projectnextnlp-chat-dialogue-corpus/json/init100/1409219608.log.json',
 '../section_4/projectnextnlp-chat-dialogue-corpus/json/init100/1408693569.log.json',
 '../section_4/projectnextnlp-chat-dialogue-corpus/json/init100/1408017260.log.json',
 '../section_4/projectnextnlp-chat-dialogue-corpus/json/init100/1407309499.log.json',
 '../section_4/projectnextnlp-chat-dialogue-corpus/json/init100/1407480626.log.json',
 '../section_4/projectnextnlp-chat-dialogue-corpus/json/init100/1408329173.log.json',
 '../section_4/projectnextnlp-chat-dialogue-corpus/json/init100/1407311921.log.json',
 '../section_4/projectnextnlp-chat-dialogue-corpus/json/init100/1407408319.log.json']

In [3]:
dialogues = []  # 複数の対話文を格納するリスト
file_count= 0  # ファイル数のカウント
for file in files:
    with open(file, "r") as f:
        json_dic = json.load(f)
        dialogue = []  # 単一の対話
        for turn in json_dic["turns"]:
            annotations = turn["annotations"]  # 注釈
            speaker = turn["speaker"]  # 発言者
            utterance = turn["utterance"]  # 発言

            # 空の文章や、特殊文字や数字が含まれる文章は除く
            if (utterance=="") or ("\\u" in utterance) or (re.search("\d", utterance)!=None):
                dialogue.clear()  # 対話をリセット
                continue

            utterance = utterance.replace(".", "。").replace(",", "、")  # 全角
            utterance = utterance.replace("．", "。").replace("，", "、")  # 半角
            utterance = utterance.split("。")[0]

            if speaker=="U":  # 発言者が人間であれば
                dialogue.append(utterance) 
            else:  # 発言者がシステムであれば
                is_wrong = False
                for annotation in annotations:
                    breakdown = annotation["breakdown"]  # 分類
                    if breakdown=="X":  # 1つでも不適切評価があれば
                        is_wrong = True
                        break
                if is_wrong:
                    dialogue.clear()  # 対話をリセット
                else:
                    dialogue.append(utterance)  # 不適切評価が無ければ対話に追加
            
            if len(dialogue) >= 2:  # 単一の会話が成立すれば
                dialogues.append(dialogue.copy())
                dialogue.pop(0)  # 最初の要素を削除

    file_count += 1
    if file_count%100 == 0:
        print("files:", file_count, "dialogues", len(dialogues))

print("files:", file_count, "dialogues", len(dialogues))

files: 100 dialogues 666
files: 200 dialogues 2111
files: 300 dialogues 3536
files: 400 dialogues 4996
files: 500 dialogues 6461
files: 600 dialogues 7939
files: 700 dialogues 9416
files: 800 dialogues 10905
files: 900 dialogues 12335
files: 1000 dialogues 13806
files: 1100 dialogues 15229
files: 1146 dialogues 15903


## データ拡張の準備
データ拡張の準備として、正規表現の設定および分かち書きを行います。

In [4]:
import re
from janome.tokenizer import Tokenizer

re_kanji = re.compile(r"^[\u4E00-\u9FD0]+$")  # 漢字の検出用
re_katakana = re.compile(r"[\u30A1-\u30F4]+")  # カタカナの検出用
j_tk = Tokenizer()

def wakati(text):
    return [tok for tok in j_tk.tokenize(text, wakati=True)] 

wakati_inp = []  # 単語に分割された入力文
wakati_rep = []  # 単語に分割された応答文
for dialogue in dialogues:
    wakati_inp.append(wakati(dialogue[0])[:10])
    wakati_rep.append(wakati(dialogue[1])[:10])

## データ拡張
対話データの数を水増しします。  
ある入力文を、それに対応する応答文以外の複数の応答文と組み合わせます。  
組み合わせる応答文は、入力文に含まれる漢字やカタカナの単語を含むものを選択します。  

In [5]:
dialogues_plus = []
for i, w_inp in enumerate(wakati_inp):  # 全ての入力文でループ
    inp_count = 0  # ある入力から生成された対話文をカウント
    for j, w_rep in enumerate(wakati_rep):  # 全ての応答文でループ
        if i==j:
            dialogues_plus.append(["".join(w_inp), "".join(w_rep)])
            continue
        similarity = 0  # 類似度
        for w in w_inp:  # 入力文と同じ単語があり、それが漢字かカタカナであれば類似度を上げる
            if (w in w_rep) and (re_kanji.fullmatch(w) or re_katakana.fullmatch(w)):
                similarity += 1
        if similarity >= 1:
            dialogue_plus = ["".join(w_inp), "".join(w_rep)]
            if dialogue_plus not in dialogues_plus:
                dialogues_plus.append(dialogue_plus)
                inp_count += 1
                if inp_count >= 12:  # ある入力から生成する対話文の上限
                    break

    if i%1000 == 0:
        print("i:", i, "dialogues_pus:", len(dialogues_plus))

print("i:", i, "dialogues_pus:", len(dialogues_plus))

i: 0 dialogues_pus: 1
i: 1000 dialogues_pus: 6448
i: 2000 dialogues_pus: 12607
i: 3000 dialogues_pus: 19802
i: 4000 dialogues_pus: 26678
i: 5000 dialogues_pus: 33402
i: 6000 dialogues_pus: 39622
i: 7000 dialogues_pus: 46109
i: 8000 dialogues_pus: 52652
i: 9000 dialogues_pus: 58914
i: 10000 dialogues_pus: 65201
i: 11000 dialogues_pus: 71769
i: 12000 dialogues_pus: 78405
i: 13000 dialogues_pus: 84787
i: 14000 dialogues_pus: 91136
i: 15000 dialogues_pus: 97211
i: 15902 dialogues_pus: 102784


拡張された対話データを、新たな対話データとします。

In [6]:
dialogues = dialogues_plus

## 対話データの保存

In [7]:
import csv
from sklearn.model_selection import train_test_split

dialogues_train, dialogues_test =  train_test_split(dialogues, shuffle=True, test_size=0.05)  # 5%がテストデータ
path = "../section_4/dialogues_data/"  # 保存場所

with open(path+"dialogues_train.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(dialogues_train)

with open(path+"dialogues_test.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(dialogues_test)

## 対話文の取得


In [8]:
import torch
import torchtext
from janome.tokenizer import Tokenizer

path = "../section_4/dialogues_data/" # 保存場所を指定

j_tk = Tokenizer()
def tokenizer(text): 
    return [tok for tok in j_tk.tokenize(text, wakati=True)]  # 内包表記
 
# データセットの列を定義
input_field = torchtext.legacy.data.Field(  # 入力文
    sequential=True,  # データ長さが可変かどうか
    tokenize=tokenizer,  # 前処理や単語分割などのための関数
    batch_first=True,  # バッチの次元を先頭に
    lower=True  # アルファベットを小文字に変換
    )

reply_field = torchtext.legacy.data.Field(  # 応答文
    sequential=True,  # データ長さが可変かどうか
    tokenize=tokenizer,  # 前処理や単語分割などのための関数
    init_token = "<sos>",  # 文章開始のトークン
    eos_token = "<eos>",  # 文章終了のトークン
    batch_first=True,  # バッチの次元を先頭に
    lower=True  # アルファベットを小文字に変換
    )
 
# csvファイルからデータセットを作成
train_data, test_data = torchtext.legacy.data.TabularDataset.splits(
    path=path,
    train="dialogues_train.csv",
    validation="dialogues_test.csv",
    format="csv",
    fields=[("inp_text", input_field), ("rep_text", reply_field)]  # 列の設定
    )

## 単語とインデックスの対応
単語にインデックスを割り振り、辞書として格納します。

In [9]:
input_field.build_vocab(
    train_data,
    min_freq=3,
    )
reply_field.build_vocab(
    train_data,
    min_freq=3,
    )

In [10]:
print(input_field.vocab.freqs)  # 各単語の出現頻度
print(len(input_field.vocab.stoi))
print(len(input_field.vocab.itos))
print(len(reply_field.vocab.stoi))
print(len(reply_field.vocab.itos))

Counter({'です': 50529, 'は': 39772, 'ね': 30960, 'が': 24274, 'か': 17831, '？': 15963, 'に': 15608, 'ます': 15170, 'の': 14373, 'を': 12163, 'よ': 11113, 'て': 10222, 'で': 8934, '、': 7011, 'ん': 6652, 'ない': 6516, 'いい': 6292, '好き': 5848, 'も': 5836, 'た': 5783, '海': 5781, 'と': 5467, 'な': 5335, 'し': 4897, 'スイカ': 4684, '！': 4534, 'だ': 4242, 'から': 3743, '行き': 3434, '退屈': 3342, 'たい': 3213, '何': 3171, '気': 3040, '症': 2757, '私': 2730, '熱中': 2666, 'ねー': 2382, 'う': 2381, 'そう': 2356, 'つけ': 2326, 'こと': 2004, '夏': 1991, 'てる': 1965, 'ねえ': 1927, 'あり': 1902, 'まし': 1860, 'こんにちは': 1758, 'い': 1705, '人': 1675, '見': 1654, '雨': 1644, 'ば': 1593, 'お': 1586, '食べ': 1530, '良い': 1466, '大丈夫': 1420, '最近': 1414, 'ませ': 1380, 'こんばんは': 1350, 'する': 1310, '楽しい': 1273, '一緒': 1271, 'とか': 1235, '朝': 1227, '多い': 1212, '今日': 1140, 'けど': 1135, '大好き': 1130, 'でしょ': 1094, '行っ': 1081, 'いえ': 1076, '美味しい': 1046, '僕': 1018, '仕事': 1016, 'へ': 990, '趣味': 946, 'ある': 943, '大事': 929, '天気': 920, 'なり': 913, '有名': 900, 'ましょ': 896, '一': 891, '方': 890, 'かも':

## データセットの保存
データセットの`examples`とFieldをそれぞれ保存します。

In [11]:
import dill

torch.save(train_data.examples, path+"train_examples.pkl", pickle_module=dill)
torch.save(test_data.examples, path+"test_examples.pkl", pickle_module=dill)

torch.save(input_field, path+"input_field.pkl", pickle_module=dill)
torch.save(reply_field, path+"reply_field.pkl", pickle_module=dill)